In [ ]:
# ------------------------------------------------------------------------------------ #
# 1stSuject_mmpUsingDict.py 1차 과제 회원관리 class + Dict 를 사용한 방법
# ------------------------------------------------------------------------------------ #

import pickle
import os
import re
from datetime import datetime

# 데이터 파일 경로 설정
DATA_FILE = '../data/members.dat'

# 에러 메시지 상수 정의
ERROR_MESSAGES = {
    "name_empty": "이름 오류: 이름을 입력해야 합니다.",
    "name_format": "이름 오류: 이름은 한글 또는 영문 1자 이상 5자 이하로 입력해야 합니다.",
    "phone_empty": "전화번호 오류: 전화번호를 입력해야 합니다.",
    "phone_format": "전화번호 오류: '000-0000-0000' 형식으로 입력해야 합니다.",
    "relation_empty": "관계 오류: 관계를 입력해야 합니다.",
    "relation_format": "관계 오류: 1, 2, 3 중 하나를 선택해야 합니다.",
    "address_empty": "주소 오류: 주소를 입력해야 합니다.",
    "duplicate_phone": "중복 오류: 이미 등록된 전화번호입니다.",
    "not_found": "검색된 목록이 없습니다.",
    "invalid_input": "잘못 입력했습니다. 다시 시도하세요.",
    "invalid_menu_number": "잘못 입력했습니다. 다시 시도하세요.",
    "exit_confirm": "정말로 종료하시겠습니까? (1. 예, 2. 아니오): ",
    "delete_confirm": "정말로 '{name}'을(를) 삭제하시겠습니까? (1. 삭제, 2. 취소하고 메인 메뉴로 돌아가기): "
}

# 메세지 상수 정의 
__MESSAGES__ = {
  'openFile' : '회원 관리 파일을 불러왔습니다.',
  'inputMenuNo' : '메뉴 번호(숫자 1~5까지)를 입력하세요: ',
  'savedData' : '회원 정보가 파일에 저장되었습니다.',
  'noData' : '등록된 회원이 없습니다.', 
  'inputReturnMainMenu' : "메인 메뉴로 돌아가려면 '<' 를 입력하세요." 
  'mainMenu' : '''
==================================================
      [ 회원관리 프로그램 메인 메뉴 ]  
--------------------------------------------------            
      아래 번호 중 하나를 입력하세요.

      1. 회원 목록 조회
      2. 회원 정보 추가(신규 등록)
      3. 회원 정보 검색 후 수정
      4. 회원 정보 검색 후 삭제
      5. 프로그램 종료
-------------------------------------------------- 
   입력 항목 * 표시 : 필수 입력 항목입니다.  
--------------------------------------------------  
'''
}



# 화면 출력용 menuTitle 
__MENU_TITLE__ = {
  '0': '[ 회원관리 프로그램 메인 메뉴 ]' ,
  '1': '[ < 메인 : 1. 회원 목록 조회 ]' ,
  '2': '[ < 메인 : 2. 회원 정보 추가(신규 등록) ]' ,
  '3': '[ < 메인 : 3. 회원 정보 검색 후 수정 ]' ,
  '4': '[ < 메인 : 4. 회원 정보 검색 후 삭제 ]' ,
}

# currentMenuNo
currentMenuNo = 0

In [ ]:
class Member:
    """개별 회원 데이터를 담는 클래스"""
    def __init__(self, name, phone, relation, address='-', reg_date=None):
        self.name = name
        self.phone = phone
        self.relation = relation
        self.address = address
        self.reg_date = reg_date if reg_date else datetime.now().strftime('%Y-%m-%d %H:%M:%S')

class MemberManager:
    """회원 전체를 관리하는 클래스 (딕셔너리 기반)"""
    def __init__(self, data_file):
        self.data_file = data_file
        self.members = {}  # 전화번호를 key로 하는 dict
        self.current_menu_no = 0
        self.load_members()


    def print_menu_title(self, menuNo) : 
        if '1' <= menuNo < '5':
            self.print_bar()
            print(__MENU_TITLE__[menuNo])
            self.print_bar()

    def print_bar(self, styleStr:str="=", nTh:int=50)->None:
        '''styleStr를 입력받아 nTh번 출력하는 함수
        param: styleStr 출력할 문자
               nTh : 반복할 횟수 로 10보다 크고 100보다 작아야 함
        유효성 검사를 통과하지 않으면 try exception하지 않고 그냥 기본값(= 50번) 출력
        return : None
        '''
        # nTh를 입력받았을때 정수형이고 100보다 작은 경우 
        if isinstance(nTh, int) and 10 < nTh <= 100:
            print(styleStr * nTh)
        else: # 그렇지 않을때 기본값 출력
            print("=" * 50)

    def load_members(self):
        """pickle로 저장된 회원 정보를 불러옴"""
        if os.path.exists(self.data_file): # 파일이 있는가?
            with open(self.data_file, 'rb') as f: # 있다면 rb 모드로 open
                try:
                    self.members = pickle.load(f) # 파일 읽어서 메모리에 로드
                except (pickle.UnpicklingError, EOFError): # 예외 처리                     
                    self.members = {} # 빈 멤버 변수 선언 
        print(__MESSAGES__['openFile']) # logging message print

    def save_members(self):
        """회원 정보를 pickle로 저장"""
        os.makedirs(os.path.dirname(self.data_file), exist_ok=True) # 파일 존재여부 확인해서 없으면 생성 just in case
        with open(self.data_file, 'wb') as f: # 파일을 쓰기 모드로 오픈
            pickle.dump(self.members, f) # 메모리에 있는걸 dump로 저장
        print(__MESSAGES__['savedData']) # logging message print

    def display_main_menu(self) -> str:
        """
            메인 메뉴 출력 및 선택
            input으로 메뉴번호를 입력받아 return 함.
        """
        print(__MESSAGES__['mainMenu'])             # 메인 메뉴 텍스트 출력 
        return input(__MESSAGES__['inputMenuNo'])   # 메뉴번호 입력 받고 리턴
           
    def list_members(self, menuNo:str):
        """전체 회원 목록 출력 및 상세 보기"""
        while True:
            self.print_menu_title(menuNo)

            if not self.members:
                print(__MESSAGES__['noData'])
                self.print_bar()
                answer = input("메인 메뉴로 돌아")
                return

            self.print_member_list(self.members)

            choice = input("상세 조회할 번호를 입력하거나, '<' 을 입력해 돌아가세요: ")
            if choice.lower() == '<':
                return
            try:
                index = int(choice) - 1
                member = list(self.members.values())[index]
                self.view_detail(member)
            except (ValueError, IndexError):
                print(ERROR_MESSAGES["invalid_input"])


In [ ]:
def main():
    """메인 루프"""
    manager = MemberManager(DATA_FILE)
    while True:
        current_menu_no = manager.display_main_menu()
        if current_menu_no == '1':
            manager.list_members(current_menu_no)            
        elif current_menu_no == '2':
            # manager.add_member(current_menu_no)
            pass
        elif current_menu_no == '3':
            # manager.update_member(current_menu_no)
            pass
        elif current_menu_no == '4':
            # manager.remove_member(current_menu_no)
            pass
        elif current_menu_no == '5':
            print(ERROR_MESSAGES['exit_confirm'] + " 상단 입력란에 입력하세요.")
            confirm = input(ERROR_MESSAGES["exit_confirm"])
            if confirm == '1':
                manager.save_members()
                print("프로그램을 종료합니다.............")
                return
        else:
            print(ERROR_MESSAGES["invalid_menu_number"])

In [29]:
main()

회원 관리 파일을 불러왔습니다.

      [ 회원관리 프로그램 메인 메뉴 ]  
--------------------------------------------------            
      아래 번호 중 하나를 입력하세요.

      1. 회원 목록 조회
      2. 회원 정보 추가(신규 등록)
      3. 회원 정보 검색 후 수정
      4. 회원 정보 검색 후 삭제
      5. 프로그램 종료
-------------------------------------------------- 
 입력 항목 * 표시 : 필수 입력 항목입니다.  
--------------------------------------------------  


[1. 회원 목록 출력]
등록된 회원이 없습니다.

      [ 회원관리 프로그램 메인 메뉴 ]  
--------------------------------------------------            
      아래 번호 중 하나를 입력하세요.

      1. 회원 목록 조회
      2. 회원 정보 추가(신규 등록)
      3. 회원 정보 검색 후 수정
      4. 회원 정보 검색 후 삭제
      5. 프로그램 종료
-------------------------------------------------- 
 입력 항목 * 표시 : 필수 입력 항목입니다.  
--------------------------------------------------  

------------------------------------------------------------
정말로 종료하시겠습니까? (1. 예, 2. 아니오):  상단 입력란에 입력하세요.
회원 정보가 파일에 저장되었습니다.
프로그램을 종료합니다.
